# A Demo of the Kriging Convolutional Network

## 1 Data loading and visualization
In this notebook, we will show how to run KCN on the bird dataset from the KCN paper. This section will load the data and visualize them on a map.

In [20]:
import pickle
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic
import os
import torch

In [21]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import data
import argument
import geopandas as gpd
import geoplot as gplt
import os
import torch
import experiment
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 180



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


## get data


In [3]:
def get_all_filenames(folder_path):
    filenames = []
    for filename in os.listdir(folder_path):
        filenames.append(filename)
    return filenames

In [4]:
folder_path = "Air_quality-Shenzhen"
all_filenames = get_all_filenames(folder_path)

In [5]:
X_raw=np.load('all.npy')
X_raw.shape

(8784, 18, 103)

In [6]:
file_result=[]
for filename in all_filenames:
    x=pd.read_csv(folder_path+"/"+filename)
    x=np.array(x)
    x=np.reshape(x,(7296, 6 ,1))
    file_result.append(x)

In [7]:
result=file_result[0]

In [8]:
for i in range(1,len(file_result)):
    result = np.append(result,file_result[i] , axis=2)

In [9]:
X_raw=result
X_raw.shape

(7296, 6, 10)

In [10]:
stations = pd.read_csv("location_of_Shenzhen.csv")

In [11]:
# X_raw=np.load('all.npy')
stations = pd.read_csv("location_of_Shenzhen.csv")
# stations = pd.read_csv("/home/dma312/Spatial_interpolation/SSIN/data/Station_info.csv")
lat=stations["lat"]
lon=stations["lon"]
lat=list(lat)
lon=list(lon)
x=X_raw.tolist()
for i in range(len(x)):
    x[i].append(lat)
    x[i].append(lon)
X=np.array(x)
for i in range(len(x)):
    X[i]=X[i].astype(np.float32)

In [12]:
X.shape

(7296, 8, 10)

In [13]:
X.shape

(7296, 8, 10)

In [14]:
xx=X[0]
xx.shape

(8, 10)

0  1  2  3  4  5 
 6  7 

In [15]:
xx=X[0]
columns_with_missing_data = np.any(np.isnan(xx), axis=0)
missing_columns = np.where(columns_with_missing_data)[0]
result = np.delete(xx, missing_columns, axis=1)
train_array = [i for i in range(int(result.shape[1]*0.8))]
test_array = [i for i in range(int(result.shape[1]*0.8),result.shape[1])]
x_train=result[[0,1,2,3,5,6,7]]
# x_train=x_train[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
x_train=x_train[:,train_array]
x_train=np.transpose(x_train)
y_train=result[4]
# y_train=y_train[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
y_train=y_train[train_array]
y_train = y_train[:, None]
x_test=result[[0,1,2,3,5,6,7]]
# x_test=x_test[:,[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
x_test=x_test[:,test_array]
x_test=np.transpose(x_test)
y_test=result[4]
# y_test=y_test[[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
y_test=y_test[test_array]
y_test = y_test[:, None]

In [16]:
args = argument.parse_opt()
args.model = 'kcn_sage'
args.dataset = "shenzhen"
args.validation_size=2

In [17]:
# args.dataset = "bird_count"
#load_ndbc_data(X_train,Y_train,X_test,Y_test,args)
trainset, testset = data.load_shenzhen_data(x_train,y_train,x_test,y_test,args)

Using the default test set from the data


In [18]:
testset.coords

tensor([[114.2475],
        [114.4240]])

### 分界点

In [22]:
err,err1 = experiment.run_kcn(x_train,y_train,x_test,y_test,args)
print('Model: {}, test error: {}\n'.format(args.model, err))

Using the default test set from the data
The shenzhen dataset has 8 training instances and 2 test instances.
Length scale is set to 0.1319656367690134


100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


Epoch: 0, train error: 0.5486071705818176, validation error: 0.24133960902690887


100%|██████████| 1/1 [00:00<00:00, 111.05it/s]


Epoch: 1, train error: 0.5256420969963074, validation error: 0.21911489963531494


...

Epoch: 22, train error: 0.17669686675071716, validation error: 0.16010338068008423


100%|██████████| 1/1 [00:00<00:00, 166.67it/s]


Epoch: 23, train error: 0.1649857461452484, validation error: 0.24279442429542542


100%|██████████| 1/1 [00:00<00:00, 125.00it/s]

Epoch: 24, train error: 0.13727155327796936, validation error: 0.17980200052261353

Early stopping at epoch 24
well done
Model: kcn_sage, test error: tensor([[0.3874],
        [0.3874]], device='cuda:0', grad_fn=<MmBackward0>)



In [10]:
loss_func=torch.nn.MSELoss(reduction="mean")

In [11]:
test_error = loss_func(err,err1)
test_error = torch.sqrt(test_error).item()
mae=torch.nn.L1Loss(reduction='mean')
test_error1=mae(err, err1).item()

### 探索多时间点怎么合并

In [34]:
# xx=X[1]
# columns_with_missing_data = np.any(np.isnan(xx), axis=0)
# missing_columns = np.where(columns_with_missing_data)[0]
# result = np.delete(xx, missing_columns, axis=1)
# train_array = [i for i in range(int(result.shape[1]*0.8))]
# test_array = [i for i in range(int(result.shape[1]*0.8),result.shape[1])]
# x_train=result[[1,2,3,4,5,6,7]]
# # x_train=x_train[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
# x_train=x_train[:,train_array]
# x_train=np.transpose(x_train)
# y_train=result[0]
# # y_train=y_train[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
# y_train=y_train[train_array]
# y_train = y_train[:, None]
# x_test=result[[1,2,3,4,5,6,7]]
# # x_test=x_test[:,[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
# x_test=x_test[:,test_array]
# x_test=np.transpose(x_test)
# y_test=result[0]
# # y_test=y_test[[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
# y_test=y_test[test_array]
# y_test = y_test[:, None]

In [35]:
# trainset, testset = data.load_shenzhen_data(x_train,y_train,x_test,y_test,args)

Using the default test set from the data


In [36]:
# err_,err1_ = experiment.run_kcn(x_train,y_train,x_test,y_test,args)

Using the default test set from the data
The shenzhen dataset has 8 training instances and 2 test instances.
Length scale is set to 0.1319656367690134


100%|██████████| 1/1 [00:00<00:00, 83.34it/s]


Epoch: 0, train error: 0.16008472442626953, validation error: 0.3570149540901184


100%|██████████| 1/1 [00:00<00:00, 83.33it/s]


Epoch: 1, train error: 0.1499241590499878, validation error: 0.20207837224006653


...

Epoch: 19, train error: 0.09417768567800522, validation error: 0.33071160316467285


100%|██████████| 1/1 [00:00<00:00, 166.67it/s]


Epoch: 20, train error: 0.05610799789428711, validation error: 0.3198240399360657


100%|██████████| 1/1 [00:00<00:00, 142.83it/s]

Epoch: 21, train error: 0.05816706269979477, validation error: 0.30599284172058105

Early stopping at epoch 21
well done


In [37]:
# err_1=torch.cat((err,err_),dim=0)

##  test结果

In [38]:
for i in range(1,len(X)):
    xx=X[i]
    # columns_with_missing_data = np.any(np.isnan(xx), axis=0)
    # missing_columns = np.where(columns_with_missing_data)[0]
    # result = np.delete(xx, missing_columns, axis=1)
    # x_train=result[[1,2,3,4,5,6,7]]
    # x_train=x_train[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
    # x_train=np.transpose(x_train)
    # y_train=result[0]
    # y_train=y_train[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
    # y_train = y_train[:, None]
    # x_test=result[[1,2,3,4,5,6,7]]
    # x_test=x_test[:,[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
    # x_test=np.transpose(x_test)
    # y_test=result[0]
    # y_test=y_test[[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
    # y_test = y_test[:, None]
    columns_with_missing_data = np.any(np.isnan(xx), axis=0)
    missing_columns = np.where(columns_with_missing_data)[0]
    result = np.delete(xx, missing_columns, axis=1)
    train_array = [i for i in range(int(result.shape[1]*0.8))]
    test_array = [i for i in range(int(result.shape[1]*0.8),result.shape[1])]
    x_train=result[[0,1,2,3,5,6,7]]
    # x_train=x_train[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
    x_train=x_train[:,train_array]
    x_train=np.transpose(x_train)
    y_train=result[4]
    # y_train=y_train[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]]
    y_train=y_train[train_array]
    y_train = y_train[:, None]
    x_test=result[[0,1,2,3,5,6,7]]
    # x_test=x_test[:,[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
    x_test=x_test[:,test_array]
    x_test=np.transpose(x_test)
    y_test=result[4]
    # y_test=y_test[[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]]
    y_test=y_test[test_array]
    y_test = y_test[:, None]
    args = argument.parse_opt()
    args.model = 'kcn_sage'
    args.dataset = "shenzhen"
    args.validation_size=2
    # trainset, testset = data.load_ndbc_data(x_train,y_train,x_test,y_test,args)
    err_,err1_ = experiment.run_kcn(x_train,y_train,x_test,y_test,args)
    err=torch.cat((err,err_),dim=0)
    err1=torch.cat((err1,err1_),dim=0)

Using the default test set from the data
The shenzhen dataset has 8 training instances and 2 test instances.
Length scale is set to 0.1319656367690134


100%|██████████| 1/1 [00:00<00:00, 66.67it/s]


Epoch: 0, train error: 0.30255478620529175, validation error: 0.3576717674732208


100%|██████████| 1/1 [00:00<00:00, 66.65it/s]


Epoch: 1, train error: 0.28672295808792114, validation error: 0.31929337978363037


...

100%|██████████| 1/1 [00:00<00:00, 117.42it/s]


Epoch: 10, train error: 0.46399176120758057, validation error: 0.0694444477558136


100%|██████████| 1/1 [00:00<00:00, 86.87it/s]


Epoch: 11, train error: 0.46399176120758057, validation error: 0.06995198875665665


100%|██████████| 1/1 [00:00<00:00, 66.54it/s]

In [ ]:
loss_func=torch.nn.MSELoss(reduction="mean")

In [ ]:
test_error = loss_func(err,err1)
test_error = torch.sqrt(test_error).item()
mae=torch.nn.L1Loss(reduction='mean')
test_error1=mae(err, err1).item()

### 分界点

In [11]:
args.model = 'kcn_sage'
err = experiment.run_kcn(args)
print('Model: {}, test error: {}\n'.format(args.model, err))

The bird_count dataset has 53623 training instances and 53623 test instances.
Length scale is set to 0.0077334818661507305


100%|██████████| 760/760 [00:23<00:00, 32.18it/s] 


Epoch: 0, train error: 0.5084476398304105, validation error: 0.5692890286445618


100%|██████████| 760/760 [00:05<00:00, 142.06it/s]


Epoch: 1, train error: 0.48033421052325714, validation error: 0.529566764831543


...

Epoch: 27, train error: 0.44300437793625813, validation error: 0.5219486951828003


100%|██████████| 760/760 [00:04<00:00, 177.13it/s]


Epoch: 28, train error: 0.4450833098325682, validation error: 0.5263857841491699


100%|██████████| 760/760 [00:04<00:00, 173.77it/s]


Epoch: 29, train error: 0.44489568774956034, validation error: 0.5291870832443237

Early stopping at epoch 29
Test error is 0.4571419060230255
Model: kcn_sage, test error: 0.4571419060230255



In [12]:
args.model = 'kcn_gat'
err = experiment.run_kcn(args)
print('Model: {}, test error: {}\n'.format(args.model, err))

The bird_count dataset has 53623 training instances and 53623 test instances.
Length scale is set to 0.0077334818661507305


100%|██████████| 760/760 [00:06<00:00, 122.74it/s]


Epoch: 0, train error: 0.5226912329757684, validation error: 0.5775631070137024


100%|██████████| 760/760 [00:06<00:00, 124.00it/s]


Epoch: 1, train error: 0.503716991858949, validation error: 0.5576544404029846


...

Epoch: 19, train error: 0.4426966260677498, validation error: 0.5017677545547485


100%|██████████| 760/760 [00:06<00:00, 123.08it/s]


Epoch: 20, train error: 0.4414260424771591, validation error: 0.5601100325584412


100%|██████████| 760/760 [00:06<00:00, 120.50it/s]


Epoch: 21, train error: 0.4462428587831949, validation error: 0.5013173222541809

Early stopping at epoch 21
Test error is 0.4536028802394867
Model: kcn_gat, test error: 0.4536028802394867



In [33]:
datafile = os.path.join(args.data_path, args.dataset + ".npz")

In [34]:
data = np.load(datafile)

In [35]:
Y_train = data['Ytrain'].astype(np.float32)

In [31]:
X_train = np.ndarray.astype(data['Xtrain'], np.float32)
Y_train = data['Ytrain'].astype(np.float32)
Y_train = Y_train[:, None]
X_test = np.ndarray.astype(data['Xtest'], np.float32)
Y_test = data['Ytest'].astype(np.float32)
Y_test = Y_test[:, None]

In [32]:
X_train.shape

(53623, 21)

In [ ]:
data = np.load(datafile)
X_train = np.ndarray.astype(data['Xtrain'], np.float32)
Y_train = data['Ytrain'].astype(np.float32)
Y_train = Y_train[:, None]
X_test = np.ndarray.astype(data['Xtest'], np.float32)
Y_test = data['Ytest'].astype(np.float32)
Y_test = Y_test[:, None]

In [23]:
trainset.coords

tensor([[ 33.6317, -84.3828],
        [ 43.2835, -72.8161],
        [ 40.7715, -74.3768],
        ...,
        [ 41.5547, -83.8526],
        [ 27.7913, -97.3992],
        [ 44.5384, -77.6191]])

In [25]:
X_train[:, 0:2].shape

(53623, 2)

In [26]:
x=[1,2,3,4,5,6,7,8,9,10]